In [ ]:
import numpy as np
import pandas as pd 

def transform(x):
    x['cp_type'] = x[x['cp_type']=='trt_cp']
    x.pop('cp_type')
    x['cp_dose'] = x['cp_dose'].replace({'D1':-1, 'D2':1})
    x['cp_time'] = x['cp_time'] // 24
    x.join(pd.get_dummies(x['cp_time']))
    x.pop('cp_time')
    return x

X_all = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
y_all = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
whatisit = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
X = transform(X_all[X_all.columns[1:]])

print([len(X_all[X_all['cp_type']==-1]),len(X)])
print([len(X_all[X_all['cp_dose']==-1]),len(X)])
print([y_all[y_all.columns[1:]].sum().sum(),whatisit[whatisit.columns[1:]].sum().sum()])
print([y_all[X_all['cp_type']==1][y_all.columns[1:]].sum().sum(),whatisit[X_all['cp_type']==1][whatisit.columns[1:]].sum().sum()])

In [ ]:
X_all = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
X_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
y_all = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
whatisit = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error,log_loss
import statistics



X_test2 = transform(X_test[X_test.columns[1:]])
X = transform(X_all[X_all.columns[1:]])

results = []

for col in y_all.columns[1:]:
    y = y_all[col]

    print(col,end=' ')
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
    model = LogisticRegression(multi_class='multinomial')
    #model = Ridge()
    model.fit(X_train, y_train)
    y_prediction = model.predict(X_val)
    y_prediction = [t if t>0.0 else 0.0 for t in list(y_prediction)]
#     results.append(log_loss(y_val,y_prediction,eps=1e-10))
    y_prediction = model.predict_proba(X_test2)
    y_prediction = [t if t>0.00001 else 0.0 for t in list(y_prediction[:,1])]
    results.append(model.coef_)
    submission[col] = y_prediction 

    

# mean = sum(results)/N
# print(mean, mean_absolute_error(results,[mean]*N))

In [ ]:
X_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
length = len(submission.columns)-1
for i in range(len(submission)): 
    if X_test.loc[i,'cp_type']!='trt_cp':
        submission.loc[i] = [submission.loc[i,'sig_id']]+[0] * length    

In [ ]:
submission.to_csv('submission.csv', index=False) #, float_format="%.10f")

In [ ]:

# import matplotlib.pyplot as plt
# import numpy as np
# n=1500
# a = pd.DataFrame(results)
# a = np.array(a)
# plt.imshow(a, cmap='hot')
# plt.show()

In [ ]:
# pd.Series(np.array(results).reshape((np.array(results).shape)[0]*(np.array(results).shape)[1])).hist(bins=21)


In [ ]:
# proportions=[submission.sum(axis=0)[1:].sum(),y_all.sum(axis=0)[1:].sum(),len(X_test2),len(y_all)]
# print(proportions)
# print(proportions[1]/proportions[3]/(proportions[0]/proportions[2]))